In [ ]:
import datetime

## Initialize Garmin api with your credentials
email = 'youremail' 
password = 'yourpassword'

## Set start date for the downloading
initial = datetime.date(year= 2023, month= 7, day=1)

In [ ]:
!pip3 install garminconnect -U

In [ ]:

from garminconnect import (
    Garmin,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
    GarminConnectAuthenticationError,
)


api = Garmin(email, password)

api.login()



In [ ]:

today = datetime.date.today()
elapsed = today - initial


bodydata =[]


for x in range(0, elapsed.days+1):
  if (x%10==0):
    print ("downloaded ",  x, " of ", elapsed.days, " days")
  bodydata.append([ initial + datetime.timedelta(days=x) , api.get_stats_and_body(initial + datetime.timedelta(days=x))])

In [ ]:
import pandas as pd


bd = pd.DataFrame([ [ bodydata[x][0], bodydata[x][1]['weight'], bodydata[x][1]['bodyFat'], bodydata[x][1]['bodyWater'], bodydata[x][1]['boneMass'], bodydata[x][1]['muscleMass'] ]  for x in range(len(bodydata))])
bd = bd.dropna()

bd.columns = ['date', 'weightkg', 'fatpercent', 'bodyWater', 'boneskg', 'musclekg' ]

bd['date'] = pd.to_datetime(bd['date'])
bd['weightkg'] =  bd['weightkg']/1000
bd['boneskg'] =  bd['boneskg']/1000
bd['musclekg'] =  bd['musclekg']/1000
bd['fatkg'] =  (bd['fatpercent'] * bd['weightkg'])/100
bd['leankg'] =  bd['weightkg'] - bd['fatkg']

display(bd)

bd.to_csv('bodydata.csv', date_format='%Y-%d-%m',  float_format='%.1f')


In [ ]:
hrvdata =[]
hr = []

for x in range(0, elapsed.days+1):
  if (x%10==0):
    print ("downloaded ",  x, " of ", elapsed.days, " days")
  try:
    hrvdata.append([ initial + datetime.timedelta(days=x) , api.get_hrv_data(initial + datetime.timedelta(days=x))])
    l = api.get_heart_rates(initial + datetime.timedelta(days=x))['heartRateValues']
    hr.append(pd.DataFrame([pd.to_datetime([ y[0] for y in l ], unit='ms'),  pd.Series( [ y[1] for y in l ] )]).T)
  except:
    pass # no hrv data

hr = pd.concat(hr)

display(hr)

In [ ]:
k  =[]
for y in range(len(hrvdata)):
  k.append(pd.DataFrame([x for x in hrvdata[y][1]['hrvReadings'] ]))

k = pd.concat(k)
k['readingTimeGMT'] = pd.to_datetime( k['readingTimeGMT'], utc=True)
k = k.rename({'hrvValue': 'rMSSD' , 'readingTimeGMT': 'timestamp_measurement' }, axis=1)

hrv = k[['rMSSD', 'timestamp_measurement']]
hr.columns = ['timestamp_measurement', 'HR']
hr['timestamp_measurement'] = pd.to_datetime(hr['timestamp_measurement'], utc=True)

hrv_final = pd.merge_asof(hrv.set_index('timestamp_measurement').sort_index(), hr.set_index('timestamp_measurement').sort_index() , left_index=True, right_index=True)
hrv_final


In [ ]:
hrv_final['HR'] =hrv_final['HR'].astype(float)
hrv_final['rMSSD'] =hrv_final['rMSSD'].astype(float)
hrv_final.reset_index(drop=False).to_csv('hrv.csv', date_format='%Y-%m-%d %H:%M:%S +0000' , index=False, float_format='%.1f')